In [1]:
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
from sklearn.cluster import MiniBatchKMeans, KMeans
## load mnist dataset
use_cuda = torch.cuda.is_available()
root = './data'
if not os.path.exists(root):
    os.mkdir(root)
#trans = transforms.Compose([transforms.Resize(32), transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
# if not exist, download mnist dataset
train_set = dset.MNIST(root=root, train=True, transform=trans, download=True)
test_set = dset.MNIST(root=root, train=False, transform=trans, download=True)
batch_size = 100
train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_size,
                 shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test_set,
                batch_size=batch_size,
                shuffle=False)

In [2]:
import time
timingResult = {}
def logTime(theName, currentTime):
    if theName not in timingResult:
        timingResult[theName] = time.time() - currentTime
    else:
        timingResult[theName] = timingResult[theName] + (time.time() - currentTime)
    currentTime = time.time()
    return currentTime

def printTiming(name):
    print('======== timing for {}: {} ======='.format(name,timingResult[name]))


In [3]:
class DEC_AE(nn.Module):
    def __init__(self, num_classes, num_features):
        super(DEC_AE,self).__init__()
        self.dropout = nn.Dropout(p=0.1)
        self.fc1 = nn.Linear(28*28,500)
        self.fc2 = nn.Linear(500,500)
        self.fc3 = nn.Linear(500,2000)
        self.fc4 = nn.Linear(2000,num_features)
        self.relu = nn.ReLU()
        self.fc_d1 = nn.Linear(500,28*28)
        self.fc_d2 = nn.Linear(500,500)
        self.fc_d3 = nn.Linear(2000,500)
        self.fc_d4 = nn.Linear(num_features,2000)
        self.alpha = 1.0
        self.clusterCenter = nn.Parameter(torch.zeros(num_classes,num_features))
        self.pretrainMode = True
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                torch.nn.init.xavier_uniform(m.weight)

    def setPretrain(self,mode):
        """To set training mode to pretrain or not, 
        so that it can control to run only the AE or AE+DECODER"""
        self.pretrainMode = mode
    
    def updateClusterCenter(self, cc):
        self.clusterCenter.data = torch.from_numpy(cc)
        
    def getTDistribution(self,x, clusterCenter):
        """ student t-distribution, as same as used in t-SNE algorithm.
         q_ij = 1/(1+dist(x_i, u_j)^2), then normalize it."""
        xe = torch.unsqueeze(x,1).cuda() - clusterCenter.cuda()
        q = 1.0 / (1.0 + (torch.sum(torch.mul(xe,xe), 2) / self.alpha))
        q = q ** (self.alpha + 1.0) / 2.0
        q = (q.t() / torch.sum(q, 1)).t() #due to divison, we need to transpose q
        return q
        
    def forward(self,x):
        x = x.view(-1, 1*28*28)
        # 32x32x1
        x = self.dropout(x)
        # 32x32x1
        x = self.fc1(x)
        # 17x17x50
        x = self.relu(x)
        # 17x17x50
        x = self.fc2(x)
        # 17x17x50
        x = self.relu(x)
        # 9x9x50
        x = self.fc3(x)
        # 17x17x50
        x = self.relu(x)
        
        x = self.fc4(x)
        
        # 9x9x50
        x_ae = x
        #if not in pretrain mode, we only need encoder
        if self.pretrainMode == False:
            return x, self.getTDistribution(x,self.clusterCenter)
        # 1x68
        ##### auto encoder is done, followed by decoder #####
        # 1x68
        x = self.fc_d4(x)
        # 1x4050
        x = self.relu(x)
        # 1x4050
        x = self.fc_d3(x)
        # 1x4050
        x = self.relu(x)
        x = self.fc_d2(x)
        # 1x4050
        x = self.relu(x)
        x = self.fc_d1(x)
        x_de = x.view(-1,1,28,28)
        # 1x4050
        return x_ae, x_de

In [4]:
import numpy as np
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score

nmi = normalized_mutual_info_score
ari = adjusted_rand_score


def acc(y_true, y_pred):
    """
    
    Calculate clustering accuracy. Require scikit-learn installed
    # Arguments
        y: true labels, numpy.array with shape `(n_samples,)`
        y_pred: predicted labels, numpy.array with shape `(n_samples,)`
    # Return
        accuracy, in [0,1]
    """
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    from sklearn.utils.linear_assignment_ import linear_assignment
    ind = linear_assignment(w.max() - w)
    return sum([w[i, j] for i, j in ind]) * 1.0 / y_pred.size

In [7]:
class DEC:
    """The class for controlling the training process of DEC"""
    def __init__(self,n_clusters,alpha=1.0):
        self.n_clusters=n_clusters
        self.alpha = alpha
        
    @staticmethod
    def target_distribution(q):
        weight = q ** 2 / q.sum(0)
        #print('q',q)
        return Variable((weight.t() / weight.sum(1)).t().data, requires_grad=True)
    def logAccuracy(self,pred,label):
        print(' '*8 + '|==>  acc: %.4f,  nmi: %.4f  <==|'
          % (acc(label, pred), nmi(label, pred)))
    def kld(self,q,p):
        res = torch.sum(p*torch.log(p/q),dim=-1)
        #print('kld ==> ', res)
        return res
    
    def validateOnCompleteTestData(self,test_loader,model):
        model.eval()
        to_eval = np.array([model(d[0].cuda())[0].data.cpu().numpy() for i,d in enumerate(test_loader)])
        true_labels = np.array([d[1].cpu().numpy() for i,d in enumerate(test_loader)])
        to_eval = np.reshape(to_eval,(to_eval.shape[0]*to_eval.shape[1],to_eval.shape[2]))
        print(to_eval.shape)
        true_labels = np.reshape(true_labels,true_labels.shape[0]*true_labels.shape[1])
        km = KMeans(n_clusters=len(np.unique(true_labels)), n_init=20, n_jobs=4)
        y_pred = km.fit_predict(to_eval)
        print(' '*8 + '|==>  acc: %.4f,  nmi: %.4f  <==|'
                      % (acc(true_labels, y_pred), nmi(true_labels, y_pred)))
        currentAcc = acc(true_labels, y_pred)
        return currentAcc
    
    def pretrain(self,train_loader, test_loader, epochs):        
        dec_ae = DEC_AE(10,10).cuda() #auto encoder
        mseloss = nn.MSELoss()
        #rint([i for i in dec_ae.parameters()])
        optimizer = optim.SGD(dec_ae.parameters(),lr = 1, momentum=0.9)
        best_acc = 0.0
        for epoch in range(epochs):
            dec_ae.train()
            running_loss=0.0
            to_eval = []
            true_labels = []
            for i,data in enumerate(train_loader):
                x, label = data
                x,label=Variable(x).cuda(),Variable(label).cuda()
                optimizer.zero_grad()
                x_ae,x_de = dec_ae(x)
                loss = F.mse_loss(x_de,x,reduce=True) 
                loss.backward()
                optimizer.step()
                x_eval = x.data.cpu().numpy()
                label_eval = label.data.cpu().numpy()
                # print statistics
                running_loss += loss.data.cpu().numpy()[0]
                if i % 100 == 99:    # print every 2000 mini-batches
                    print('[%d, %5d] loss: %.7f' %
                          (epoch + 1, i + 1, running_loss / 100))
                    #print('x_de:',x_de, x)
                    running_loss = 0.0
            #now we evaluate the accuracy with AE
            dec_ae.eval()
            currentAcc = self.validateOnCompleteTestData(test_loader,dec_ae)
            if currentAcc > best_acc:                
                torch.save(dec_ae,'bestModel'.format(best_acc))
                best_acc = currentAcc
                

    
    def train(self,train_loader, test_loader, epochs):
        """This method will start training for DEC cluster"""
        ct = time.time()
        model = torch.load("bestModel").cuda()
        model.setPretrain(False)
        optimizer = optim.SGD([\
             {'params': model.parameters()}, \
            ],lr = 0.001, momentum=0.9)
        print('Initializing cluster center with pre-trained weights')
        km = MiniBatchKMeans(n_clusters=self.n_clusters, n_init=20, batch_size=batch_size)
        got_cluster_center = False
        for epoch in range(epochs):
            running_loss=0.0
            for i,data in enumerate(train_loader):
                x, label = data
                x = Variable(x).cuda()
                optimizer.zero_grad()
                #step 1 - get cluster center from batch
                if not got_cluster_center:                
                    model.eval()
                    y_pred_ae,_ = model(x)
                    #print('y_pred',y_pred_ae)                    
                    y_pred_ae = y_pred_ae.data.cpu().numpy()
                    #print('ae prediction', y_pred_ae.shape)
                    y_pred = km.partial_fit(y_pred_ae) #seems we can only get a centre from batch
                    #print('cluster center:',km.cluster_centers_.shape)
                    self.cluster_centers = km.cluster_centers_ #keep the cluster centers
                    model.updateClusterCenter(self.cluster_centers)
                    #rint('model',model.state_dict())
                    if epoch > 1:
                        got_cluster_center = True
                else:
                    model.train()
                    #now we start training with acquired cluster center
                    feature_pred,q= model(x)
                    #get target distribution
                    p = self.target_distribution(q)
                    #print('q',q,'p',p)
                    loss = self.kld(q,p).mean()
                    loss.backward()
                    optimizer.step()
                    running_loss = running_loss + loss.data.cpu().numpy()[0]
            currentAcc = self.validateOnCompleteTestData(test_loader,model)


In [8]:
#now start training
import random
random.seed(7)
dec = DEC(10)
dec.pretrain(train_loader, test_loader, 50)
dec.train(train_loader, test_loader, 100)

[1,   100] loss: 0.0779668
[1,   200] loss: 0.0595135
[1,   300] loss: 0.0483055
[1,   400] loss: 0.0406491
[1,   500] loss: 0.0361709
[1,   600] loss: 0.0338063
(10000, 10)
        |==>  acc: 0.5194,  nmi: 0.4881  <==|


/private/xlia/anaconda3/envs/py35/lib/python3.5/site-packages/torch/serialization.py:159: UserWarning: Couldn't retrieve source code for container of type DEC_AE. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


[2,   100] loss: 0.0317056
[2,   200] loss: 0.0307140
[2,   300] loss: 0.0291085
[2,   400] loss: 0.0282583
[2,   500] loss: 0.0274381
[2,   600] loss: 0.0270275
(10000, 10)
        |==>  acc: 0.5581,  nmi: 0.5278  <==|
[3,   100] loss: 0.0261021
[3,   200] loss: 0.0258156
[3,   300] loss: 0.0253449
[3,   400] loss: 0.0247645
[3,   500] loss: 0.0244093
[3,   600] loss: 0.0242366
(10000, 10)
        |==>  acc: 0.5831,  nmi: 0.5517  <==|
[4,   100] loss: 0.0237356
[4,   200] loss: 0.0234857
[4,   300] loss: 0.0232656
[4,   400] loss: 0.0231485
[4,   500] loss: 0.0225917
[4,   600] loss: 0.0225319
(10000, 10)
        |==>  acc: 0.5944,  nmi: 0.5638  <==|
[5,   100] loss: 0.0223180
[5,   200] loss: 0.0219938
[5,   300] loss: 0.0217699
[5,   400] loss: 0.0216775
[5,   500] loss: 0.0212407
[5,   600] loss: 0.0213630
(10000, 10)
        |==>  acc: 0.6206,  nmi: 0.5839  <==|
[6,   100] loss: 0.0210852
[6,   200] loss: 0.0209596
[6,   300] loss: 0.0206662
[6,   400] loss: 0.0206058
[6,   500] l

In [ ]:
a=[[1,2],[2,3]]
np.stack(a)